In [ ]:
#| export
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

from sklearn.preprocessing import StandardScaler
from pathlib import Path

In [ ]:
#| export
def collect_snps(method):
    """
    Input
        method: a Path(PosixPath) to directory containing npy arrays for each chr
    Output
        tuple(hybrid ids, snp matrix)
    """
    for c,chr in enumerate(method.iterdir()):
        if c == 0:
            strains,snp_data = np.load(chr,allow_pickle=True)
        else:
            strains, snps = np.load(chr, allow_pickle=True)
            snp_data = np.vstack((snp_data,snps))

            return strains,snp_data

In [ ]:
#| export
class WT():
    """
    A class which will hold the weather data for the entire dataset for training purposes
    
    init
        weather_data -> pandas table
        testYear -> e.g. 2019. this will set all data from a given year as the Test Set
    """
    def __init__(self, weather_data, testYear):
        
        self.Te = weather_data.iloc[([str(testYear) in x for x in weather_data['Year']]),:].reset_index()
        self.Tr = weather_data.iloc[([str(testYear) not in x for x in weather_data['Year']]),:].reset_index()
            
        self.setup_scaler()
        self.scale_data(self.Tr)
        self.scale_data(self.Te)
            
    def setup_scaler(self):
        ss = StandardScaler()
        ss.fit(self.Tr.select_dtypes('float'))
        self.scaler = ss
            
    def scale_data(self, df):
        fd = df.select_dtypes('float')
        fs = self.scaler.transform(fd)
        df[fd.columns] = fs

In [ ]:
#| export
class YT():
    """
    A class which will hold the yield data for the entire dataset for training purposes
    
    init
        yield_data -> pandas table
        testYear -> e.g. 2019. this will set all data from a given year as the Test Set
    """
    def __init__(self, yield_data, testYear):

        self.Te = yield_data.iloc[([str(testYear) in x for x in yield_data['Env']]),:].reset_index()
        self.Tr = yield_data.iloc[([str(testYear) not in x for x in yield_data['Env']]),:].reset_index()

        self.setup_scaler()
        self.scale_data(self.Tr)
        self.scale_data(self.Te)

    def setup_scaler(self):
        ss = StandardScaler()
        ss.fit(np.array(self.Tr['Twt_kg_m3']).reshape(-1,1))
        self.scaler = ss

    def scale_data(self,df):
        ya = np.array(df['Twt_kg_m3']).reshape(-1,1)
        ys = self.scaler.transform(ya)
        df['scaled_yield'] = ys

    def plot_yields(self):

        plt.hist(self.Tr['scaled_yield'],density=True, label='Train',alpha=.5,bins=50)
        plt.hist(self.Te['scaled_yield'],density=True, label='Test',alpha=.5,bins=50)
        plt.legend()
        plt.show()

In [ ]:
#| export
class GEM():
    """
    init
        split -> the year that will be designated as the test split
    """
    def __init__(self, split):
        self.split = str(split)
        self.Y = None
        self.W = None
        self.SNP = None

In [ ]:

#| export
class GemDataset():
    """
    Pytorch Dataset which can be used with dataloaders for simple batching during training loops
    """
    def __init__(self,W,Y,G):
        self.W = W
        self.SNP = G
        self.Y = Y
        
    def __len__(self): return self.Y.shape[0]

    def __getitem__(self,idx):
        #keys to access SNPs and Weather
        hybrid = self.Y.iloc[idx,:]['Hybrid']
        env = self.Y.iloc[idx,:]['Env']

        #values for the model training
        target = self.Y.iloc[idx,:]['scaled_yield']
        genotype = self.SNP[1][:, np.where(self.SNP[0]==hybrid)[0][0]]
        weather = np.array(self.W.loc[self.W['Env'] == env].select_dtypes('float'))

        return target, genotype, weather 

In [ ]:
#| hide
def remove_leapdays(weather):
    """ just a hotfix """
    to_remove = []
    for i in list(set(weather_data['Env'])):
        if (sum(weather_data['Env'] == i)) == 366:
            #get indexes
            to_remove.append(max(list(weather_data.loc[weather_data['Env'] == i].index)))
    return weather_data.drop(to_remove)

In [ ]:
test_split = 2019
path_snps = Path('data/snpCompress/')
data_path = Path('data/')
path_train_weatherTable =data_path/'Training_Data/4_Training_Weather_Data_2014_2021.csv'
path_train_yieldTable = data_path/'Training_Data/1_Training_Trait_Data_2014_2021.csv'
snp_compression = 'PCS_50'
batch_size = 64

In [ ]:
snp_data = collect_snps(path_snps/snp_compression) # Read in the SNP profiles
yield_data = pd.read_csv(path_train_yieldTable) # Read in trait data 
yield_data = yield_data[yield_data['Yield_Mg_ha'].notnull()] #Remove plots w/ missing yields
weather_data = pd.read_csv(path_train_weatherTable) # Read in Weather Data
weather_data['Year'] = [x.split('_')[1] for x in weather_data['Env']] #Store Year in a new column
#removes yield data where no weather data
setYield = set(yield_data['Env'])
setWeather = set(weather_data['Env'])
only_yield = setYield - setWeather
only_weather = setWeather - setYield
yield_data = yield_data.iloc[[x not in only_yield for x in yield_data['Env']],:]
#removes yield data where no genotype data
setSNP = set(snp_data[0])
setYield = set(yield_data['Hybrid'])
only_yield = setYield - setSNP
yield_data = yield_data.iloc[[x not in only_yield for x in yield_data['Hybrid']],:]

weather_data = remove_leapdays(weather_data)
weather_data = weather_data.reset_index()
yield_data=yield_data.sample(frac=1)
yield_data = yield_data.reset_index()

In [ ]:
import torch
import torch.nn as nn

# Define the model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        return x

# Instantiate the model
model = MLP(input_size=100, hidden_size=64, output_size=1)

# Generate some random input data
input_data = torch.randn(32, 100)

# Make a prediction
output = model(input_data)

In [ ]:
from tqdm import tqdm
from torch.utils.data import DataLoader
from torch import tensor,nn,optim
import torch.nn.functional as F


In [ ]:

#Create a GEM dataset
gem = GEM(test_split)

gem.Y = YT(yield_data, test_split)
gem.W = WT(weather_data, test_split)
gem.SNP = snp_data

tr_ds = GemDataset(gem.W.Tr, gem.Y.Tr, gem.SNP)
te_ds = GemDataset(gem.W.Te, gem.Y.Te, gem.SNP)

tr_dataloader = DataLoader(tr_ds, batch_size=batch_size, shuffle=True)
te_dataloader = DataLoader(te_ds, batch_size=batch_size, shuffle=True)

dls = (tr_dataloader, te_dataloader)


In [ ]:

losses = []
model = MLP(100,100,1)
opt = torch.optim.SGD(model.parameters(), lr=.05)
loss_func = F.mse_loss
for c, (y,g,w) in enumerate(tr_dataloader):
    g = g.type(torch.float32)
    #print(g.shape)
    preds = model(g)
    y = y.type(torch.float32)
    loss = loss_func(preds.squeeze(),y)
    loss.backward()
    opt.step()
    opt.zero_grad()
    #print(preds[0])
    losses.append(loss.detach().numpy())
    
    if preds[0].isnan()[0] == True:
        print('nan')
    
    if c > 500:
        print(c)
        break
    

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [ ]:
import matplotlib.pyplot as plt
plt.plot(losses)